# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2022-2023, Q3
* Course homepage: https://brightspace.tudelft.nl/d2l/home/500969
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Ruben Martin Rodriguez (R.MartinRodriguez@student.tudelft.nl)
* (c) TU Delft, 2023

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [ ]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = ""
STUDENT_1_STUDENT_NUMBER = ""
STUDENT_1_NETID = ""
STUDENT_1_EMAIL = ""

In [ ]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions, and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for youthat you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Task 2d.2 - PD-ILC (11p)

**Author:** Maximilian Stölzle (M.W.Stolzle@tudelft.nl)

This task focuses on tuning the sequence of feedforward actions using PD-ILC to track a trajectory as well as possible. We assume the following scenario: for control purposes, we only have have access to a perturbed set of robot parameters. Specifically, the mass and inertia are increased with respect to the nominal robot model. Therefore, we observe a significant tracking error when using a PD + pure feedforward controller (see Task 2a.3) and simulating the closed-loop system using the nominal robot parameters. The purpose of applying ILC to this problem setting is to iteratively decrease this tracking error by tuning the feed-forward control action. It needs to be stressed that the controller never has access to the nominal dynamical model and thus the perturbed parameters are also used for the linearization of the system.

In [ ]:
# Reloads the python files outside of this notebook automatically
%load_ext autoreload
%autoreload 2

from distutils.util import strtobool
from functools import partial
from IPython.display import display, HTML  # For animations in the notebook
from jax.config import config as jax_config

jax_config.update("jax_platform_name", "cpu")  # set default device to 'cpu'
jax_config.update("jax_enable_x64", True)  # double precision
from jax import debug, jacfwd, jit, lax, vmap
import jax.numpy as jnp
from jax.scipy import linalg
import os
from pathlib import Path
from typing import Callable, Dict, Tuple

# define boolean to check if the notebook is run for the purposes of autograding
AUTOGRADING = strtobool(os.environ.get("AUTOGRADING", "false"))

## Implementation of PD-ILC (4p)

We model the trajectory to be discretized into $N$ time-steps, each of duration $\delta t$. In the following we will conform to the super-vector / lifted system notation as introduced during the lectures: $\theta_j(k) \in \mathbb{R}^2$ for example corresponds to the link angles at the $k$th time-step and at the $j$th ILC iteration. $Y_j \in \mathbb{R}^{2 (N-1)}$ describes the system output sequence in super-vector notation.

But first, we need to implement a small utility function. Namely, `blk_diag` should take a stack of block matrices (i.e. a tensor of shape $(m, n, o)$, where $(n, o)$ is the block matrix) and place them along the diagonal of a matrix. This resulting block-diagonal matrix will then have the shape $(m \: n, m \: o)$.

This exercise is a good opportunity to get familiar with the [`jax.lax.fori_loop`](https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.fori_loop.html) and [`jax.lax.dynamic_update_slice`](https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.dynamic_update_slice.html), which allow for jittable `for` loops and array _slicing_ respectively. This will make your function run significantly faster compared to naive, purely Pythonic implementation, which will come in handy later for Q-ILC.

In [ ]:
@jit
def blk_diag(a: jnp.ndarray) -> jnp.ndarray:
    """
    Create a block diagonal matrix from a tensor of blocks
    Args:
        a: matrices to be block diagonalized of shape (m, n, o)

    Returns:
        b: block diagonal matrix of shape (m * n, m * o)

    """

    def assign_block_diagonal(i, _b):
        """
        Save the ith block  into the block-diagonal matrix `_b`
        Args:
            i: Index of block which we save into the block-diagonal matrix.
            _b: Block diagonal matrix. Should still have zeros at the ith block.
        Returns
        """
        # Assign the block saved in ith entry of `a` to the ith block-diagonal of `_b`
        # Hint: use `jax.lax.dynamic_update_slice` to update the entries of `_b`
        # YOUR CODE HERE
        raise NotImplementedError()
        return _b

    # Implement for loop to assign each block in `a` to the block-diagonal of `b`
    # Hint: use `jax.lax.fori_loop` and pass `assign_block_diagonal` as an argument
    b = jnp.zeros((a.shape[0] * a.shape[1], a.shape[0] * a.shape[2]))
    # YOUR CODE HERE
    raise NotImplementedError()

    return b

In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL

# define block entry
_a_block = jnp.array(
    [
        [1.0, 2.0, 0.0],
        [-2.0, 1.0, 4.0],
    ]
)

# we repeat `_a_block` along the 0-dimension to get _a, which has shape (4, 2, 3)
_a = jnp.repeat(jnp.expand_dims(_a_block, axis=0), 4, axis=0)

# resulting block-diagonal matrix `_b` should have shape (8, 12)
_b = blk_diag(_a)

print("Block diagonal matrix:\n", _b)

assert _b.shape[0] == _a.shape[0] * _a.shape[1]
assert _b.shape[1] == _a.shape[0] * _a.shape[2]


Now, please implement the learning rule for PD-ILC

\begin{equation}
    u_{j+1} = u_j(k) + k_\mathrm{p,ILC}(k) \: e_j(k+1) + k_\mathrm{d,ILC}(k) \: [e_j(k+1) - e_j(k)]
\end{equation}

where $u \in \mathbb{R}^2$ is the ILC action and $e_j(k) = y^\mathrm{d}(k) - y_j(k) \in \mathbb{R}^2$ is the error between the desired output and the actual system output.
The same written in super-vector notation looks like

\begin{equation}
    U_{j+1} = U_{j} + K_\mathrm{p,ILC} \: E + K_{\mathrm{d,ILC}} \: \delta E
\end{equation}

where now $U, E, \delta E \in \mathbb{R}^{2 (N-1)}$ and $K_\mathrm{p,ILC}, K_\mathrm{d,ILC} \in \mathbb{R}^{2 (N-1) \times 2 (N-1)}$.

In [ ]:
@jit
def learning_rule_pd_ilc(
    u_ts: jnp.ndarray,
    y_ts: jnp.ndarray,
    y_des_ts: jnp.ndarray,
    Kp_ilc: jnp.ndarray,
    Kd_ilc: jnp.ndarray,
) -> jnp.array:
    """
    Implements the PD-ILC learning rule to compute the ILC control action `U_next` for the next iteration

    Args:
        u_ts: array of shape (N, 2) containing the ILC actions at the current iteration
        y_ts: array of shape (N, 2) containing the system outputs at the current iteration
        y_des_ts: array of shape (N, 2) containing the desired system outputs
        Kp_ilc: proportional gain of shape (2 * (N - 1), 2 * (N - 1))
        Kd_ilc: derivative gain  of shape (2 * (N - 1), 2 * (N - 1))

    Returns:
        u_nextit_ts: array of shape (N, 2) containing the ILC actions at the next iteration
    """
    # number of time-steps
    N = u_ts.shape[0]

    # extract the last (N-1) time-steps of the outputs and transform to super-vector notation
    Y = jnp.zeros((2 * (N - 1)))
    Y_des = jnp.zeros_like(Y)
    # YOUR CODE HERE
    raise NotImplementedError()

    # compute the output error E of the last (N-1) time-steps in super-vector notation
    E = jnp.zeros_like(Y)
    # YOUR CODE HERE
    raise NotImplementedError()

    # compute the change in error between time-steps in super-vector notation.
    # the change in error should be computed considering **all* time-steps,
    # resulting in an array `delta_E` of shape (2 * (N-1), )
    delta_E = jnp.zeros_like(E)
    # YOUR CODE HERE
    raise NotImplementedError()

    # extract the first (N-1) time-steps of the ILC actions and transform to super-vector notation
    U = jnp.zeros((2 * (N - 1)))
    # YOUR CODE HERE
    raise NotImplementedError()

    # implement the PD learning rule and compute the ILC action at the next iteration `U_nextit`
    # should be array of shape (2 * (N-1), ) in super-vector notation
    U_nexit = jnp.zeros_like(U)
    # YOUR CODE HERE
    raise NotImplementedError()

    # transform `U_nexit` from super-vector notation to array `u_nexit_ts` of shape (N, 2)
    # Hint: the control action at the last time-step is always zero
    # as it does not have an influence on the system states within the given time horizon
    u_nextit_ts = jnp.zeros_like(u_ts)
    # YOUR CODE HERE
    raise NotImplementedError()

    return u_nextit_ts

In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL
_y_ts = jnp.arange(10, dtype=jnp.double).reshape((-1, 2))
_y_des_ts = jnp.arange(start=10, step=-1, stop=0, dtype=jnp.double).reshape((-1, 2))
_u_ts = jnp.arange(0, 10, dtype=jnp.double).reshape((5, 2))
_N = _y_ts.shape[0]  # number of time-steps
_Kp_ilc = jnp.eye(2 * (_N - 1))
_Kd_ilc = 0.1 * jnp.eye(2 * (_N - 1))

_u_nextit_ts = learning_rule_pd_ilc(_u_ts, _y_ts, _y_des_ts, _Kp_ilc, _Kd_ilc)
print("Computed ILC action at the next iteration:\n", _u_nextit_ts)

# target for next ILC actuation sequence
_u_nextit_ts_target = jnp.array(
    [
        [5.6, 4.6],
        [3.6, 2.6],
        [1.6, 0.6],
        [-0.4, -1.4],
        [0.0, 0.0],
    ]
)
print("Target ILC action at the next iteration:\n", _u_nextit_ts)
# make sure that the results of your implementation matches the target
assert jnp.allclose(_u_nextit_ts_target, _u_nextit_ts)


Next, please study the implementation of the functions. `init_ilc_its` and `apply_ilc_control_action_to_system` in `ilc.py` to understand the structure of the `ilc_its` dictionary, which stores the system states, inputs and outputs across iterations and simulation time. Afterwards, please implement the `run_pd_ilc` and `pd_ilc_iteration` function below to execute PD-ILC. The main steps are:

- Linearizing the system about the given trajectory.
- Computing the proportional and derivative gain matrices for the ILC algorithm `Kp_ilc` and `Kd_ilc` in super-vector notation. At each time-step $k$, the PD-ILC gain matrices should be equal to $K_{\mathrm{p,ILC}}(k) = k_{\mathrm{p,ILC}} \, (C_\mathrm{d}(k) \: B_\mathrm{d}(k))^{-1}$, where $k_{\mathrm{p,ILC}} \in \mathbb{R}$, $k_{\mathrm{d,ILC}} \in \mathbb{R}$ are scalar gains and $B_\mathrm{d}(k)$, $C_\mathrm{d}(k)$ are part of the discretized, linear state-space representation.
- Computing the feedforward torque sequence $\tau_\mathrm{ilc} \in \mathbb{R}^{N \times 2}$ to be applied to the system.
- Defining the error $E_j$ between the desired system output sequence $Y^\mathrm{d}_j$ and the actual system output sequence $Y_j$. 
- Determining the ILC action sequence at the next iteration $U_{j+1}$ by implementing the PD-ILC learning rule. Following the usual convention, please make sure to only update the first $N-1$ entries of the ILC control action. Similarily, the learning rule should only consider the last $N-1$ entries of $E \in \mathbb{R}^{N}$. Within the learning rule, we set the filter $Q$ to the identity matrix 


In [ ]:
from ilc import init_ilc_its, apply_ilc_control_action_to_system

# import linearize_closed_loop_fb_system_about_trajectory from linearization.ipynb
from ipynb.fs.full.linearization import linearize_closed_loop_fb_system_about_trajectory


def run_pd_ilc(
    rp: dict,
    traj_ts: Dict[str, jnp.ndarray],
    th_0: jnp.ndarray,
    th_d_0: jnp.ndarray,
    num_iterations: int,
    kp_ilc: float = 0.0,
    kd_ilc: float = 0.0,
    kp_fb: jnp.ndarray = jnp.zeros((2, 2)),
    kd_fb: jnp.ndarray = jnp.zeros((2, 2)),
    rp_perturbed: dict = None,
):
    """
    Run the PD-ILC algorithm to track a desired trajectory
    Args:
        rp: dictionary of robot parameters
        traj_ts: dictionary of time series of trajectories
        th_0: initial link angles of shape (2,)
        th_d_0: initial link angular velocities of shape (2,)
        num_iterations: number of iterations of the Q-ILC algorithm
        kp_ilc: proportional gain
        kd_ilc: derivative gain
        kp_fb: proportional gains of the parallel feedback controller of shape (2, 2)
        kd_fb: derivative gains of the parallel feedback controller of shape (2, 2)
        rp_perturbed: dictionary of perturbed robot parameters used for linearizing the model and computing the
            trajectory equilibrium torque. If not specified, the nominal robot parameters are used.
    Returns:
        ilc_its: dictionary to track states across iterations and time steps
    """
    # number of time-steps
    N = traj_ts["t_ts"].shape[0]

    # define perturbed robot parameters used for linearizing the robot
    if rp_perturbed is None:
        rp_perturbed = rp.copy()

    # linearize the system about the trajectory with the corresponding equilibrium states and torques
    #    - tau_eq_ts is the compute equilbrium torque
    #    - Ad_ts, Bd_ts, Cd_ts, Dd_ts are the resulting discrete-time, time-varying, linear state-space matrices
    # It is not allowed to use `rp` here, as the controller is not allowed to have access to the
    # nominal robot model.
    # Hint: use a function from the `linearization.ipynb` notebook.
    tau_eq_ts = jnp.zeros((N, 2))
    Ad_ts, Bd_ts = jnp.zeros((N, 4, 4)), jnp.zeros((N, 4, 2))
    Cd_ts, Dd_ts = jnp.zeros((N, 2, 4)), jnp.zeros((N, 2, 2))
    # YOUR CODE HERE
    raise NotImplementedError()

    # we estimate the ILC control action initially to be zero
    # the torque at the last time index is always zero and never applied to the system
    u_ts = jnp.zeros_like(traj_ts["th_ts"])

    # initialize the dictionary to save iterations
    ilc_its = init_ilc_its(num_iterations, traj_ts)
    ilc_its["tau_eq_ts"] = tau_eq_ts
    ilc_its["u_nextit_ts"] = u_ts

    # Compute the PD-ILC gains Kp_ilc and Kd_ilc
    # These gains are used to optimize `u_ts` after each ILC iteration
    # Hint: make use of the `blk_diag` function and
    # the discretized, time-varying, linear state-space representation
    Kp_ilc = kp_ilc * jnp.zeros((2 * (N - 1), 2 * (N - 1)))
    Kd_ilc = kd_ilc * jnp.zeros_like(Kp_ilc)

    # YOUR CODE HERE
    raise NotImplementedError()

    # this is the main loop for iterations
    ilc_its = lax.fori_loop(
        0,  # lower bound
        num_iterations,  # upper bound
        partial(
            pd_ilc_iteration,
            rp,
            traj_ts,
            th_0,
            th_d_0,
            tau_eq_ts,
            Kp_ilc,
            Kd_ilc,
            kp_fb,
            kd_fb,
        ),  # function to be called
        ilc_its,  # init_val
    )

    # remove u_ts_next_it entries only needed for the loop
    ilc_its.pop("u_nextit_ts")

    return ilc_its


@jit
def pd_ilc_iteration(
    rp: dict,
    traj_ts: Dict[str, jnp.ndarray],
    th_0: jnp.ndarray,
    th_d_0: jnp.ndarray,
    tau_eq_ts: jnp.ndarray,
    Kp_ilc: jnp.ndarray,
    Kd_ilc: jnp.ndarray,
    kp_fb: jnp.ndarray,
    kd_fb: jnp.ndarray,
    it: int,
    ilc_its: Dict[str, jnp.ndarray],
) -> Dict[str, jnp.ndarray]:
    """
    Perform one iteration of the PD-ILC algorithm

    Args:
        rp: dictionary of robot parameters
        traj_ts: dictionary of time series of trajectories
        th_0: initial link angles of shape (2,)
        th_d_0: initial link angular velocities of shape (2,)
        tau_eq_ts: time series of equilibrium torques of shape (N, 2)
        Kp_ilc: proportional gain of shape (2 * (N - 1), 2 * (N - 1))
        Kd_ilc: derivative gain  of shape (2 * (N - 1), 2 * (N - 1))
        kp_fb: proportional gains of the parallel feedback controller of shape (2, 2)
        kd_fb: derivative gains of the parallel feedback controller of shape (2, 2)
        it: iteration index
        ilc_its: dictionary to track states across iterations and time steps
    Returns:
        ilc_its: updated dictionary to track states across iterations and time steps
    """
    # Read-out the ILC action `u_ts` computed at the end of the last iteration
    u_ts = ilc_its["u_nextit_ts"]
    ilc_its["u_its"] = ilc_its["u_its"].at[it].set(u_ts)

    # Compute the feedforward torque sequence tau_ilc_ts as a sum of
    # the equilbrium torque and the ILC control action
    tau_ilc_ts = jnp.zeros_like(tau_eq_ts)
    # YOUR CODE HERE
    raise NotImplementedError()

    # simulate the system with the current control action
    sim_ts, ilc_its = apply_ilc_control_action_to_system(
        rp=rp,
        traj_ts=traj_ts,
        th_0=th_0,
        th_d_0=th_d_0,
        it=it,
        ilc_its=ilc_its,
        tau_ilc_ts=tau_ilc_ts,
        kp_fb=kp_fb,
        kd_fb=kd_fb,
    )

    # Call the PD-ILC learning rule
    # Hint: the sequence of link angles is stored in the `traj_ts` and `sim_ts` dictionaries.
    u_nextit_ts = jnp.zeros_like(u_ts)
    # YOUR CODE HERE
    raise NotImplementedError()

    # Store the ILC action for the next iteration in the dictionary
    ilc_its["u_nextit_ts"] = u_nextit_ts

    return ilc_its

## Let's run it (7p)

Now, let's actually run PD-ILC. As mentioned before, we will use the perturbed robot parameters to derive the control law(s) and the nominal robot model to simulate the double pendulum. The perturbed model namely assumes larger link masses and inertias.

For the PD feedback controller, we set the gains to $k_\mathrm{p} = \mathrm{diag}(500, 500)$ Nm / rad and $k_\mathrm{d} = \mathrm{diag}(50, 50)$ Nm s / rad. Please note that you are **not** allowed to change these gains. Instead, please tune the PD-ILC gains, which are denoted with `kp_ilc` and `kd_ilc`, and the number of ILC iterations to achieve good tracking performance defined as **the Euclidean norm of the RMSE in operational-space being below 0.06 m.**

In [ ]:
from jax_double_pendulum.analysis import *
from jax_double_pendulum.motion_planning import (
    generate_ellipse_trajectory,
    ELLIPSE_PARAMS,
)
from jax_double_pendulum.robot_parameters import ROBOT_PARAMS
from jax_double_pendulum.robot_simulation import simulate_robot
from jax_double_pendulum.utils import normalize_link_angles
from jax_double_pendulum.visualization import animate_robot

from ilc_analysis import (
    plot_configuration_space_ilc_convergence,
    animate_configuration_space_trajectory_following_plot,
    animate_operational_space_trajectory_following_plot,
    animate_actuation_plot,
)

# the pertubation factor determines by how much the masses and inertia of the perturbed robot model
# used for control is increased with respect to the actual / nominal systems (i.e. in our case used for simulation)
perturbation_factor = 3

# robot parameters
rp = ROBOT_PARAMS

# perturbed robot parameters used for linearizing the system
rp_perturbed = rp.copy()
rp_perturbed.update(
    {
        "m1": perturbation_factor * rp["m1"],
        "j1": perturbation_factor * rp["j1"],
        "m2": perturbation_factor * rp["m2"],
        "j2": perturbation_factor * rp["j2"],
    }
)

# simulation parameters
sim_duration = 10.0  # [s]
sim_dt = 1e-2  # [s]

# define time steps
t_ts = sim_dt * jnp.arange(int(sim_duration / sim_dt))

# gains of parallel feedback PD controller
# IMPORTANT: you are not allowed to change the feedback PD gains!!!
kp_fb = 500 * jnp.eye(2)  # [Nm/rad]
kd_fb = 50 * jnp.eye(2)  # [Nm s/rad]

# generate ellipse trajectory
traj_ts = generate_ellipse_trajectory(
    rp=rp,
    t_ts=t_ts,
    **ELLIPSE_PARAMS,
)

# specify the initial link angles at the beginning of each simulation
th_0 = traj_ts["th_ts"][0] - jnp.array([0.1, 0.2])

In [ ]:
# number of ILC iterations
# please tune appropiately
num_ilc_iterations = 10

# ILC-PD controller gains
# please tune appropiately
kp_ilc = 0e0  # [Nm/rad]
kd_ilc = 0e0  # [Nm s/rad]

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
ilc_its = run_pd_ilc(
    rp=rp,
    traj_ts=traj_ts,
    num_iterations=num_ilc_iterations,
    th_0=th_0,
    th_d_0=traj_ts["th_d_ts"][0],
    kp_ilc=kp_ilc,
    kd_ilc=kd_ilc,
    kp_fb=kp_fb,
    kd_fb=kd_fb,
    rp_perturbed=rp_perturbed,
)

# define folder where to save animations and plots
outputs_dir = Path("outputs")
outputs_dir.mkdir(parents=True, exist_ok=True)

# plot configuration-space ILC convergence
plot_configuration_space_ilc_convergence(
    traj_ts,
    ilc_its,
    show=True,
    filepath=str(outputs_dir / "task_2d-2_ilc_convergence.pdf"),
)

In [ ]:
# animate the configuration-space evolution through the iterations
if not AUTOGRADING:
    ani_configuration_space = animate_configuration_space_trajectory_following_plot(
        traj_ts,
        ilc_its,
        max_num_animated_its=50,
        show=False,
        filepath=str(
            outputs_dir / "task_2d-2_ilc_configuration_space_trajectory_following.mp4"
        ),
    )
    display(HTML(ani_configuration_space.to_html5_video()))

In [ ]:
# animate the operational-space evolution through the iterations
if not AUTOGRADING:
    ani_operational_space = animate_operational_space_trajectory_following_plot(
        traj_ts,
        ilc_its,
        max_num_animated_its=50,
        show=False,
        filepath=str(
            outputs_dir / "task_2d-2_ilc_operational_space_trajectory_following.mp4"
        ),
    )
    display(HTML(ani_operational_space.to_html5_video()))

In [ ]:
# animation the actuation sequence through the iterations
if not AUTOGRADING:
    ani_actuation = animate_actuation_plot(
        traj_ts,
        ilc_its,
        max_num_animated_its=50,
        show=False,
        filepath=str(outputs_dir / "task_2d-2_ilc_actuation.mp4"),
    )
    display(HTML(ani_actuation.to_html5_video()))

In [ ]:
# import feedback controller from controllers.ipynb
from ipynb.fs.full.controllers import ctrl_fb_pd

# First, extract the feed-forward torques from the last ILC iteration.
# Then, simulate the system again using those torques and a PD feedback controller
# and save the system states to `sim_ts`.
# Hint: the feed-forward torques of shape (num_ilc_its, num_timesteps, 2) are accessible via
# `ilc_its["tau_ilc_its"]` and should be passed to the `tau_ext_ts` argument of the simulation method.
# YOUR CODE HERE
raise NotImplementedError()

# plot the configuration-space evolution
plot_configuration_space_trajectory_following(
    traj_ts,
    sim_ts,
    filepath=str(
        outputs_dir / "task_2d-2_configuration_space_trajectory_following.pdf"
    ),
)

In [ ]:
rmse_th, rmse_th_d, rmse_th_dd = compute_configuration_space_rmse(traj_ts, sim_ts)
rmse_x, rmse_x_d, rmse_x_dd = compute_operational_space_rmse(traj_ts, sim_ts)
with jnp.printoptions(precision=3):
    print(
        "RMSE theta:",
        rmse_th,
        "rad, RMSE theta_d:",
        rmse_th_d,
        "rad/s, RMSE theta_dd:",
        rmse_th_dd,
        "rad/s^2",
    )
    print(
        "RMSE x:",
        f"{jnp.linalg.norm(rmse_x):.4f}",
        "m, RMSE x_d:",
        f"{jnp.linalg.norm(rmse_x_d):.3f}",
        "m/s, RMSE x_dd:",
        f"{jnp.linalg.norm(rmse_x_dd):.2f}",
        "m/s^2",
    )

In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL

grader_total = 7
rmse_x_lb = 0.06  # m --> this will result in the full grade
rmse_x_ub = 1.00  # m --> this will result in a zero grade
grader_points = grader_total * (
    (1 - (jnp.linalg.norm(rmse_x) - rmse_x_lb) / (rmse_x_ub - rmse_x_lb))
)
grader_points = jnp.clip(grader_points, 0, grader_total).item()

print(
    f"If you submit the assignment as it is, you will receive {grader_points} points."
)


grader_points

In [ ]:
# animate this simulation
if not AUTOGRADING:
    ani_robot = animate_robot(
        rp_perturbed,
        traj_ts=traj_ts,
        sim_ts=sim_ts,
        step_skip=5,
        show=False,
        filepath=str(outputs_dir / "task_2d-2_controlled_robot.mp4"),
    )
    display(HTML(ani_robot.to_html5_video()))